# 1. 사전 세팅

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설치
!sudo apt-get install -y fonts-nanum -qq
!sudo fc-cache -f
!rm -rf ~/.cache/matplotlib

%cd "/content/drive/MyDrive/데이터 분석/projects/ML_protfoilo/1. 의류 판매량 예측 모델링"

# !gdown --folder "data_link" --remaining-ok

In [ ]:
!pwd

# 2. 문제 정의


---

    A 의류 회사에서 생산량을 조절하기 위해 판매량을 예측하려고 한다.
    의류 종류에 따라 계절성을 타기 때문에, 이런 특징을 고려할 수 있는 모델링이 필요하다.



## 2.1. 도메인 지식

---

▸ 티셔츠 제조의 Lead Time(주문부터 납품까지 걸리는 전체 시간)

    티셔츠 발주를 넣어 생산하는 데 통상 2~3주가 걸린다.
    그러나, 보통 베트남이나 중국 등의 해외에서의 생산을 주로 하기 때문에 유통 시간까지 고려해서 계산해야 한다.
    배에 싣거나, 검사하거나, 이동하는 시간까지 고려하게 되면 총 8주 정도가 걸린다고 예상한다.

▸ 재고 비용

    재고를 가지고 있을 때 발생하는 비용으로, 재고가 생기면 창고에 적재되어야 하므로 창고 사용료, 운영료 등을 줄이는 것을 목적으로 해야 한다.
    그렇기에 생산량을 조절해 재고를 최소화하는 것이 매우 중요하다.

▸ MD의 역할

    MD가 작년 판매량과 회사의 기대치(의지치)를 반영해 발주를 하게 된다.
    이를 정할 때에는 오직 MD의 감으로 Baseline + α을 잡게 된다.
    
▸ Demand Plan(수요 계획)

    보통 한 번에 몇 만 장을 발주하는 것이 아니라, 나눠서 발주를 하게 된다.
    이때 사고가 나는 경우가 많은데, 적게 주문했을 때 재고가 부족한 점, 사람들이 트렌드가 바뀌어 사지 않는 점 등이 드러날 수 있다.

▸ 트렌드의 변화

    옷차림 트렌드는 날씨에 따라 가장 빈번하게 바뀌고, 대형 포털의 검색어에 따라서도 바뀔 수 있다.

# 3. 데이터 확인

## 3.1. 데이터 명세

---

    데이터는 종류/날짜별로 한 행씩 구분되어 총 500개 가량의 서로 다른 티셔츠가 제공되어 있다.

| Column | Description |
|:----|:----|
| SEASON | SS(Spring/Summer) or AW(Autumn/Winter) |
| Type | 국내, 해외 등 |
| ... | 날짜 등 |

## 3.2. 데이터 살펴보기

In [ ]:
%cd "/content/drive/MyDrive/데이터 분석/projects/ML_protfoilo/1. 의류 판매량 예측 모델링"
!ls

In [ ]:
# pytorch-forecasting: PyTorch DeepAR
# MXNet 기반이 호환 문제로 Lightning 기반 DeepAR로 진행
# !pip install pytorch-forecasting==1.5.0 pytorch-lightning==2.4.0 --quiet

# 결국 다 안되서... darts로 해봄
!pip install "darts>=0.30.0,<0.37.0" --quiet

In [ ]:
import pandas as pd

# Default가 max_columns은 20, max_rows는 60이니까 늘려서 안 짤리게 하기
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

df = pd.read_excel('Data/WEAR_TS_ALL.xlsx')
df.head()

    티셔츠마다 판매 시작 일자가 다름을 확인했다.
    각각의 SKU(재고 관리 단위)에 대한 예측을 하게 되면 시간도 오래 걸리고, 데이터의 양도 적어 비효율적이다.
    그렇기에 이들을 따로 보는 것이 아닌 하나로 합쳐 분석할 것이다.


▸ 왜도와 첨도

    왜도(7일 동안의 평균의 왜도)를 확인하면 현재 판매량의 트렌드를 확인할 수 있다.
    첨도가 높게 되면 쏠림 현상이 일어나기 때문에, 특정 이벤트가 있었음을 짐작할 수 있다.

In [ ]:
# 515개의 제품군(SKU;Stock Keeping Unit)에 대해 1463개의 행을 보유
df.shape

## 3.3. 데이터 가공 명세

---

| 명세 | 내용 |
|:----|:----|
| 가공 범위 | <input type="checkbox" checked> Feature(Column) 변형<br> <input type="checkbox">기간 단위 데이터 분석 및 Big Volume 특성을 반영하기 위한 데이터 증강<br> <input type="checkbox" checked>다양한 Table을 참조하기 위한 데이터 분할|
| 가공 상세 | <b>Feature(Column) 변형</b><br><br>- 열로 구성되어 있는 날짜 정보를 Transpose하여 행으로 변환<br>- 단변량 데이터에서 정보 취득을 위해(Random Forest 구축을 위해) 각종 통계량으로 변환<br><br><b>데이터 분할</b><br><br>- 검증을 위해 학습 95%, 검증 5%로 분할 |
| 가공 결과 | <b>As is</b><br><br>- row: 515 / column: 1463 / table: 1<br><br> <b>To be(1)</b><br><br>- row: 1461(+946) / column: 1(-1462) / table: 1<br><br> <b>To be(2)</b><br><br>- row: 1308(-153) / column: 15(+14) / table: 1  |

# 4. 문제 해결 프로세스 정의

▸ 문제

    A 회사에서 재고에 대한 걱정이 늘어나고 있다.
    판매량을 알 수 있다면 재고 비용을 비약적으로 줄일 수 있을 것으로 기대하고 있다.
    
▸ 기대 효과

    의류 시장에서의 큰 비용 중에 하나는 재고 비용이다. 적정 생산량을 통해 재고 비용을 줄일 수 있다.

▸ 해결 방안

    시계열 예측과 Random Forest를 통한 판매량 예측
    시계열 분석을 위해서는 데이터를 단변량 데이터로 변환해야 한다.
    Random Forest를 위해서는 다변량 데이터로 변환해야 한다.

▸ 성과 측정

    실제 판매량과 예측량의 차이

▸ 현업 적용

    시계열 특성을 가진 데이터(보통 세상 모든 데이터는 시계열)를 만났을 때, 접근할 수 있는 방법 체득

# 5. 데이터 전처리 및 EDA

## 5.1. 데이터 전처리

In [ ]:
# 필요한 패키지 Import
import os
from pathlib import Path

import matplotlib.pyplot as plt
# numpy와의 호환 문제(np.bool 삭제)로 mxnet import 불가
# import mxnet as mx
import numpy as np
import pandas as pd
from matplotlib import rc
from matplotlib.dates import date2num

rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

# mx.random.seed(0)
np.random.seed(0)

In [ ]:
df = pd.read_excel('Data/WEAR_TS_ALL.xlsx')
df

In [ ]:
# 20년도 데이터의 부족 문제로 20년 데이터는 제외 예정
len(df[df['SEASON'] == '20SS'])

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
# 국내와 해외는 특성 차이로 인해 동시에 모델링 불가하기에 국내 데이터만 필터링
df = df[df['TYPE'] == '국내']
df.drop('TYPE', axis=1, inplace=True)

# 시즌별 판매량 예측이 목적이기 때문에 Season별로 group by하여 합계 생성
df = df.groupby('SEASON').sum()
df

In [ ]:
# 날짜별 판매량을 행으로 다루기 위한 전치
df = df.T

# 현재 Index가 날짜형 Index가 아니기 때문에 날짜형으로 변경
df.index = pd.to_datetime(df.index)
df

In [ ]:
df.index

In [ ]:
# 하나의 열로 병합
df = pd.concat([
    df[df.index.year == 2016]['16SS'],
    df[df.index.year == 2017]['17SS'],
    df[df.index.year == 2018]['18SS'],
    df[df.index.year == 2019]['19SS']
])

df = pd.DataFrame(df, columns=['sales'])
df

In [ ]:
df.plot(figsize=(20, 7))
plt.show()

    이상치가 있음을 확인했다.
    그리고 마이너스가 찍힐 수는 있지만(환불로 인한), 2017년 중반부와 같은 마이너스는 말이 되지 않는다.
    중국 큰 손들(?)의 장난질일 수 있음 - 도메인 지식을 갖춘 이들과의 인터뷰가 이래서 중요하다.

In [ ]:
# 시스템 오류로 인한 Outlier 삭제를 위해 값을 치환
df[(df['sales'] < -1000) | (df['sales'] > 2500)] = 0

In [ ]:
df.plot(figsize=(20, 7))
plt.show()

    특이점들의 대한 분석과 의문이 들어야 데이터를 해석할 수 있다.
    
    보통 확 증가하는 부분에서는 프로모션이 진행했음을 알 수 있고,
    보통 확 감소하는 부분에서 재고 비용이 많이 발생하게 된다.


In [ ]:
# 학습 데이터는 2019년 여름 시즌 이전까지
train_df = df[:-200]

# test 데이터는 2019년 여름 시즌부터 겨울 시즌 전까지 Peak 100일에 대한 예측
test_df = df[-200:-100]

In [ ]:
train_df.plot(figsize=(20, 7))
plt.show()

In [ ]:
test_df.plot(figsize=(20, 7))
plt.show()

    판매량이 확 떨어지는 구간에 대해 예측을 해야 그 구간에서 더 이상 생산하지 않는다.
    그래서 6월의 판매량을 예측하게 되면, 그 8주 전인 4월의 발주량을 결정할 수 있게 된다.

# 6. 시계열 예측 모델링

In [ ]:
from darts import TimeSeries
from darts.models import RNNModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.likelihood_models import GaussianLikelihood
from darts.metrics import mape, mae

In [ ]:
series = TimeSeries.from_series(df['sales'])

In [ ]:
# 학습/검증 데이터 분리
train_size = int(len(series) * 0.9)
train, val = series[:train_size], series[train_size:]

# 스케일링
scaler = Scaler()
train_scaled = scaler.fit_transform(train)
val_scaled = scaler.transform(val)

In [ ]:
# RNN 기반 예측 모델 정의 (현대식 DeepAR 스타일)
model = RNNModel(
    model="LSTM",                 # GluonTS의 cell_type='lstm'
    input_chunk_length=30,        # context_length=30
    training_length=60,           # input_chunk_length + 예측 길이 정도
    hidden_dim=40,                # num_cells=40
    n_rnn_layers=4,               # num_layers=4
    dropout=0.1,                  # GluonTS 기본 0.1
    batch_size=64,                # num_batches_per_epoch=80 과 유사한 효과
    n_epochs=30,                  # epochs=30
    optimizer_kwargs={"lr": 1e-3},# learning_rate=1e-3
    likelihood=GaussianLikelihood(),  # 확률적 예측
    random_state=42,
    force_reset=True,
    pl_trainer_kwargs={
        "accelerator": "auto",           # GPU 자동
        "enable_checkpointing": False,
        "logger": False
    }
)

In [ ]:
# 모델 학습
model.fit(train_scaled, verbose=True)

In [ ]:
forecast_scaled = model.predict(n=30, series=train_scaled)
forecast = scaler.inverse_transform(forecast_scaled)

In [ ]:
# 모델 평가
print("MAPE:", mape(val, forecast))
print("MAE :", mae(val, forecast))

In [ ]:
# 시각화
plt.figure(figsize=(10, 4))
series.plot(label="actual")
forecast.plot(label="forecast")
plt.axvline(series.time_index[train_size], color="gray", linestyle="--")
plt.title("의류 판매량 예측 (RNNModel)")
plt.legend()
plt.show()